In [0]:
# https://towardsdatascience.com/facial-recognition-using-deep-learning-a74e9059a150

In [0]:
import dlib
import numpy as np
import os
from matplotlib.pyplot import imread
import imageio
from tqdm import tqdm_notebook

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# !pip install scipy==1.4.1
# scipy.__version__

In [0]:
!ls

himanshu  obama  saanika


In [0]:
cd /content/gdrive/My Drive/face_recognition/

/content/gdrive/My Drive/face_recognition


In [0]:
ls

 dlib_face_recognition_resnet_model_v1.dat
 DSC_1907.jpg
 DSC_1915.jpg
 DSC_1921.jpg
 face_recog.ipynb
 images/
 shape_predictor_68_face_landmarks.dat
 test/
 train/
'WhatsApp Image 2020-02-08 at 1.32.08 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.32.34 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.33.17 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.33.32 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.35.48 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.38.16 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.38.28 AM.jpg'
'WhatsApp Image 2020-02-08 at 1.40.09 AM.jpg'
 ZDSC_1908.jpg
 ZDSC_1909.jpg


In [0]:
# Get Face Detector from dlib
# This allows us to detect faces in images
face_detector = dlib.get_frontal_face_detector()
# Get Pose Predictor from dlib
# This allows us to detect landmark points in faces and understand the pose/angle of the face
shape_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
# Get the face recognition model
# This is what gives us the face encodings (numbers that identify the face of a particular person)
face_recognition_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
# This is the tolerance for face comparisons
# The lower the number - the stricter the comparison
# To avoid false matches, use lower value
# To avoid false negatives (i.e. faces of the same person doesn't match), use higher value
# 0.5-0.6 works well
TOLERANCE = 0.4

In [0]:
# This function will take an image and return its face encodings using the neural network
def get_face_encodings(path_to_image):
    # Load image using scipy
    image = imageio.imread(path_to_image)
    # Detect faces using the face detector
    detected_faces = face_detector(image, 1)
    # Get pose/landmarks of those faces
    # Will be used as an input to the function that computes face encodings
    # This allows the neural network to be able to produce similar numbers for faces of the same people, regardless of camera angle and/or face positioning in the image
    shapes_faces = [shape_predictor(image, face) for face in detected_faces]
    # For every face detected, compute the face encodings
    return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shapes_faces]

In [0]:
# This function takes a list of known faces
def compare_face_encodings(known_faces, face):
    # Finds the difference between each known face and the given face (that we are comparing)
    # Calculate norm for the differences with each known face
    # Return an array with True/Face values based on whether or not a known face matched with the given face
    # A match occurs when the (norm) difference between a known face and the given face is less than or equal to the TOLERANCE value
    return (np.linalg.norm(known_faces - face, axis=1) <= TOLERANCE)

In [0]:
# This function returns the name of the person whose image matches with the given face (or 'Not Found')
# known_faces is a list of face encodings
# names is a list of the names of people (in the same order as the face encodings - to match the name with an encoding)
# face is the face we are looking for
def find_match(known_faces, names, face):
    # Call compare_face_encodings to get a list of True/False values indicating whether or not there's a match
    matches = compare_face_encodings(known_faces, face)
    # Return the name of the first match
    count = 0
    for match in matches:
        if match:
            # print("names=",names)
            return names[count].split('/')[1]
            
        count += 1
    # Return not found if no match found
    return 'Not Found'

In [0]:
%%time
# Get path to all the known images
# Filtering on .jpg extension - so this will only work with JPEG images ending with .jpg
face_encodings = []
names = []
my_list =['saanika','himanshu']
for z in my_list:
    image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir('/content/gdrive/My Drive/face_recognition/train/' + z + '/'))
    # Sort in alphabetical order
    image_filenames = sorted(image_filenames)
    # Get full paths to images
    paths_to_images = ['train/' + z + '/' + x for x in image_filenames]
    # List of face encodings we have
    print(paths_to_images)
    # Loop over images to get the encoding one by one
    for path_to_image in tqdm_notebook(paths_to_images):
        
        # Get face encodings from the image
        f = 0
        face_encodings_in_image = get_face_encodings(path_to_image)
        # Make sure there's exactly one face in the image
        if len(face_encodings_in_image) != 1:
            print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
            f = 1
            # exit()
        # Append the face encoding found in that image to the list of face encodings we have
        if(f == 0):
            names.append(path_to_image)
            face_encodings.append(get_face_encodings(path_to_image)[0])
            # face_encodings.append(face_encodings_in_image(path_to_image)[0])

['train/saanika/DSC_1915.jpg', 'train/saanika/WhatsApp Image 2020-02-08 at 1.32.08 AM.jpg', 'train/saanika/WhatsApp Image 2020-02-08 at 1.32.34 AM.jpg', 'train/saanika/WhatsApp Image 2020-02-08 at 1.33.17 AM.jpg', 'train/saanika/WhatsApp Image 2020-02-08 at 1.35.48 AM.jpg', 'train/saanika/WhatsApp Image 2020-02-08 at 1.38.28 AM.jpg', 'train/saanika/WhatsApp Image 2020-02-08 at 1.40.09 AM.jpg', 'train/saanika/ZDSC_1908.jpg', 'train/saanika/ZDSC_1909.jpg']


Please change image: train/saanika/DSC_1915.jpg - it has 3 faces; it can only have one
Please change image: train/saanika/ZDSC_1908.jpg - it has 2 faces; it can only have one


KeyboardInterrupt: ignored

In [0]:
# Get path to all the test images
# Filtering on .jpg extension - so this will only work with JPEG images ending with .jpg
test_filenames = filter(lambda x: x.endswith(('.jpg','.jpeg','JPG')), os.listdir('test/'))
# Get full paths to test images
paths_to_test_images = ['test/' + x for x in test_filenames]
# Get list of names of people by eliminating the .JPG extension from image filenames
# names = [x[:-4] for x in image_filenames]
# Iterate over test images to find match one by one
for path_to_image in tqdm_notebook(paths_to_test_images):
    # Get face encodings from the test image
    face_encodings_in_image = get_face_encodings(path_to_image)
    # Make sure there's exactly one face in the image
    if len(face_encodings_in_image) == 0:
        print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
        # exit()
    elif(len(face_encodings_in_image) == 1):
    # Find match for the face encoding found in this test image
        match = find_match(face_encodings, names, face_encodings_in_image[0])
        print(path_to_image, match)
    else:
        for x in range(len(face_encodings_in_image)):
            match = find_match(face_encodings, names, face_encodings_in_image[0])
            # Print the path of test image and the corresponding match
            print(path_to_image, match)
    print()

test/IMG_2834.JPG himanshu

test/IMG_2957.JPG himanshu

test/IMG_2980.JPG himanshu

test/IMG_2986.JPG himanshu

test/IMG_2990.JPG himanshu

Please change image: test/IMG_20160324_145540417_HDR.jpg - it has 0 faces; it can only have one

test/IMG_20160324_151532386.jpg himanshu

test/IMG_20160324_151855277 (1).jpg himanshu

Please change image: test/IMG_2936_01.jpg - it has 0 faces; it can only have one

Please change image: test/IMG_2948_01.jpg - it has 0 faces; it can only have one

test/IMG_7444.jpg saanika
test/IMG_7444.jpg saanika

test/IMG_7448.jpg saanika

test/IMG_7450.jpg saanika

test/IMG_7568.jpg saanika

test/IMG_7571.jpg saanika

test/IMG_7578.jpg saanika
test/IMG_7578.jpg saanika

test/IMG_7579__01.jpg himanshu

test/IMG_7655__01.jpg saanika

test/IMG_7657__01.jpg saanika

test/00000008.jpg Not Found

test/00000009.jpg Not Found

test/00000003.jpg Not Found

test/00000000.jpg Not Found

test/DSC_1907.jpg saanika
test/DSC_1907.jpg saanika



KeyboardInterrupt: ignored

In [0]:
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
 
    # Display the resulting frame
    cv2.imshow('Frame',frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))